In [99]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime as dt
from pytz import timezone
from IPython.display import display
import numpy as np


In [100]:
# Load in the merged dataframe

ET = 'Canada/Eastern'
start_time = dt.now(timezone(ET))
today = str(start_time)[:10]
inputFileDate = today
# inputFileDate = '2022-07-19'

mergedFilePathName = f'/home/jovyan/ODBiz/3-Merging/output/4-ODBiz_merged_{inputFileDate}.csv'
total_lines = 802383
chunksize = 10000
# df = pd.read_csv(mergedFilePathName, low_memory=False)
df = pd.concat([chunk for chunk in tqdm(pd.read_csv(mergedFilePathName, chunksize=chunksize, dtype=str), 
                                        desc='Loading data', 
                                        total=total_lines//chunksize+1)
                ])
num_of_rows = df.shape[0]

print(f'Successfully loaded {mergedFilePathName}')

Loading data: 100%|██████████| 81/81 [00:06<00:00, 12.70it/s]
Successfully loaded /home/jovyan/ODBiz/3-Merging/output/4-ODBiz_merged_2022-07-28.csv


In [101]:
print(num_of_rows)
pd.options.display.max_columns = None
df.head()

802383


,idx,localfile,business_name,business_sector,business_subsector,business_description,business_id_no,licence_number,licence_type,primary_NAICS,secondary_NAICS,NAICS_descr,alt_econ_act_code,alt_econ_act_descrip,latitude,longitude,full_address,full_address_2,mailing_address,unit,street_no,street_name,street_direction,street_type,city,province,postal_code,country,business_website,email,telephone,telephone_extension,toll_free_telephone,fax,total_no_employees,no_full_time,no_part_time,no_seasonal,date_established,indigenous,status,provider,geo_source,valid_coord,addr_info,has_addr_info,invalid_NAICS_query
0,1352c00083568dfebe65,BC_Victoria_Business_Licences.csv,KURVA DESIGN,INTERMUNICIPAL - MOBILE SERVICES,NaN,Construction / Specialty trade contractors,NaN,25071,NaN,NaN,NaN,NaN,NaN,NaN,48.4310107,-123.3412079,1417 PEMBROKE ST VICTORIA BC V8R 1V7,NaN,NaN,NaN,1417,PEMBROKE ST,NaN,NaN,Victoria,BC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,City of Victoria,Source,True,1417 PEMBROKE ST VICTORIA BC V8R 1V7____1417_P...,True,NaN
1,85b2584ba2eaa631d414,BC_Victoria_Business_Licences.csv,PARACHUTE ICE CREAM,RESTAURANT - GENERAL,NaN,Accommodation & food services / Food services ...,NaN,31145,NaN,NaN,NaN,NaN,NaN,NaN,48.43653971,-123.3734922,105-2626 BRIDGE ST VICTORIA BC V8T 4S9,NaN,NaN,105,2626,BRIDGE ST,NaN,NaN,Victoria,BC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,City of Victoria,Source,True,105-2626 BRIDGE ST VICTORIA BC V8T 4S9___105_2...,True,NaN
2,21307e1f41eb27227271,BC_Victoria_Business_Licences.csv,PIC-A-FLIC VIDEO,RENTALS & SALES - VARIOUS,NaN,Retail trade / Misc. store retailers,NaN,30415,NaN,NaN,NaN,NaN,NaN,NaN,48.42598245,-123.3389962,1519 PANDORA AVE VICTORIA BC V8R 6P9,NaN,NaN,NaN,1519,PANDORA AVE,NaN,NaN,Victoria,BC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,City of Victoria,Source,True,1519 PANDORA AVE VICTORIA BC V8R 6P9____1519_P...,True,NaN
3,abb26454eabba5f3ee70,BC_Victoria_Business_Licences.csv,CRAIGDARROCH CASTLE HISTORICAL MUSEUM SOCIETY,NON PROFIT SOCIETY - GENERAL,NaN,Other services (exc. Public administration) / ...,NaN,34109,NaN,NaN,NaN,NaN,NaN,NaN,48.4230248,-123.3436825,1070 JOAN CRES VICTORIA BC V8S 3L5,NaN,NaN,NaN,1070,JOAN CRES,NaN,NaN,Victoria,BC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,City of Victoria,Source,True,1070 JOAN CRES VICTORIA BC V8S 3L5____1070_JOA...,True,NaN
4,2ff1e557764ea6205464,BC_Victoria_Business_Licences.csv,INSUREBIZZ FINANCIAL INC,BUSINESS SERVICES - FINANCIAL,NaN,Finance & insurance / Insurance carriers & rel...,NaN,29261,NaN,NaN,NaN,NaN,NaN,NaN,48.42598245,-123.3389962,1540 FORT ST VICTORIA BC V8S 5J2,NaN,NaN,NaN,1540,FORT ST,NaN,NaN,Victoria,BC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,City of Victoria,Source,True,1540 FORT ST VICTORIA BC V8S 5J2____1540_FORT ...,True,NaN


In [102]:
# NAICS Codes that return an empty query
max_rows = 100
pd.options.display.max_rows = max_rows

naics_df = df[['idx', 'localfile', 'primary_NAICS']]
naics_df = naics_df.dropna(subset = ['primary_NAICS'])


# # Display only entries with a trailing 0
naics_df['primary_NAICS'] = naics_df['primary_NAICS'].str.replace('.0', '', regex = False)
naics_df['primary_NAICS'] = naics_df['primary_NAICS'].replace('TBD', np.nan, regex = False)
naics_df['primary_NAICS'] = naics_df['primary_NAICS'].replace('tbd', np.nan, regex = False)
naics_df['primary_NAICS'] = naics_df['primary_NAICS'].replace('-', np.nan, regex = False)
naics_df['primary_NAICS'] = naics_df['primary_NAICS'].str.replace('\s\-\s.*', '', regex = True)

naics_df = naics_df.dropna(subset = ['primary_NAICS'])
# display(naics_df.loc[793827])
# display(naics_df[(naics_df['localfile'] == 'Indigenous_Business_Directory.csv') & (naics_df['primary_NAICS'].str.contains('-', regex = False))])

naics_df['primary_NAICS'] = naics_df['primary_NAICS'].astype('Int64')
naics_df = naics_df[naics_df['primary_NAICS'] % 10 == 0]
naics_df = naics_df[naics_df['primary_NAICS'] // 100000 == 0]

# with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):  # more options can be specified also
# print(naics_df.loc[8066, 'localfile'])
display(naics_df.iloc[-max_rows:])
# pd.unique(naics_df)

,idx,localfile,primary_NAICS
681188,5db06b87af86d5a895aa,ON_York_Region_Business_Directory.csv,0
681191,8ba730197c3dffcbff8a,ON_York_Region_Business_Directory.csv,0
681358,530c1c1c81516205e85b,ON_York_Region_Business_Directory.csv,0
681574,69fd1acdb9dcb9582d89,ON_York_Region_Business_Directory.csv,0
682000,4aa2abbdf212e367b63c,ON_York_Region_Business_Directory.csv,0
682376,c3decc80440c07173b71,ON_York_Region_Business_Directory.csv,0
682480,934e5fb43378b3ba5d3a,ON_York_Region_Business_Directory.csv,0
682482,b38e70716972ec32bc17,ON_York_Region_Business_Directory.csv,0
682521,80f6b5023e892f987aa7,ON_York_Region_Business_Directory.csv,0
682736,54628774b586e9d967c6,ON_York_Region_Business_Directory.csv,0


In [103]:
x = pd.unique(df['primary_NAICS'].dropna()).astype(int)
x = x[~np.isnan(x)]
x.sort()
for i in x:
    print(i)

0
1
11
21
22
23
31
32
33
41
42
44
45
48
49
51
52
53
54
55
56
61
62
71
72
81
91
113
236
237
238
313
315
316
321
322
323
325
326
331
332
333
334
336
337
339
413
414
415
416
417
418
441
442
443
444
445
446
447
448
451
452
453
454
482
484
485
488
491
492
493
511
512
517
519
522
523
524
526
531
532
533
541
561
562
611
621
623
624
711
713
722
811
812
813
814
912
2361
2362
2381
2382
2383
2389
3152
3256
3323
3331
3335
3339
4189
4411
4421
4422
4441
4451
4452
4453
4471
4481
4482
4513
4521
4529
4531
4533
4539
4541
4841
4842
4882
4883
4921
4922
5111
5112
5122
5173
5179
5191
5239
5242
5312
5321
5411
5412
5415
5416
5418
5511
5611
5613
5622
5629
6111
6112
6113
6115
6116
6117
6211
6212
6215
6216
6231
6239
6244
7139
7213
7224
7225
8111
8112
8113
8114
8122
8131
8133
8134
8139
11111
11121
11133
11141
11142
11199
11211
11232
11292
11299
11511
11521
21111
21222
21232
21311
22111
22112
22121
22131
22132
22133
23611
23621
23622
23711
23712
23713
23721
23731
23799
23811
23812
23813
23814
23815
23816
23817
238

In [104]:
df[df['primary_NAICS'] == '-812910']

,idx,localfile,business_name,business_sector,business_subsector,business_description,business_id_no,licence_number,licence_type,primary_NAICS,secondary_NAICS,NAICS_descr,alt_econ_act_code,alt_econ_act_descrip,latitude,longitude,full_address,full_address_2,mailing_address,unit,street_no,street_name,street_direction,street_type,city,province,postal_code,country,business_website,email,telephone,telephone_extension,toll_free_telephone,fax,total_no_employees,no_full_time,no_part_time,no_seasonal,date_established,indigenous,status,provider,geo_source,valid_coord,addr_info,has_addr_info,invalid_NAICS_query


In [105]:
err

NameError: name 'err' is not defined

In [4]:
# The two businesses with weird province codes
x = df[df['province'].isin(['78', 'LAKEVIEW ESTATES'])]
# x.iloc[1]
x

,idx,localfile,business_name,business_sector,business_subsector,business_description,business_id_no,licence_number,licence_type,primary_NAICS,secondary_NAICS,NAICS_descr,alt_econ_act_code,alt_econ_act_descrip,latitude,longitude,full_address,full_address_2,mailing_address,unit,street_no,street_name,street_direction,street_type,city,province,postal_code,country,business_website,email,telephone,telephone_extension,toll_free_telephone,fax,total_no_employees,no_full_time,no_part_time,no_seasonal,date_established,indigenous,status,provider,geo_source,valid_coord,addr_info,has_addr_info
83338,7a122c15a250341d251a,BC_Vancouver_Business_Licences.csv,Post The Butcher Inc,Office,Broker,NaN,NaN,19-138327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vancouver,78,NaN,CANADA,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,no,Gone Out of Business,City of Vancouver,NaN,False,________Vancouver_,True
166464,69139e8fad512fe66792,BC_Vancouver_Business_Licences.csv,(Naoko Winther),Health Services,Counselling,NaN,NaN,22-123793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20 - 07,LAKEVIEW ESTATES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,no,Issued,City of Vancouver,NaN,False,________20 - 07_,True


In [5]:
# Verify that all source files are present in the merged file and extract a list of non-Canadian 'provinces'

print(f'Number of unique data sources: {len(pd.unique(df.localfile))}')
print('--------------------------------------------------------------')
print('Unique Non-Canadian Province Entries:')
can_prov_lst = ['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'NT', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT']
provinces = pd.unique(df.province)
non_cad_provs = []
for i in provinces:
    if not(i in can_prov_lst):
        non_cad_provs.append(i)
        print(i)




Number of unique data sources: 56
--------------------------------------------------------------
Unique Non-Canadian Province Entries:
nan
78
LAKEVIEW ESTATES


In [6]:
# Extract a list of indices corresponding to non-Canadian entries and obtain the subset of the df that contains these entries
non_cad_idx= []
for i, row in tqdm(df.iterrows(), total = num_of_rows):
    if row.province in non_cad_provs:
        non_cad_idx.append(i)
print(f'Number of non-Canadian entries found: {len(non_cad_idx)}')

non_cad_df = df.loc[non_cad_idx]
nonCanadianEntriesFileName = '/home/jovyan/ODBiz/3-Merging/double_check/nonCanadianEntries.csv'
non_cad_df.to_csv(nonCanadianEntriesFileName)
print(f'{nonCanadianEntriesFileName} saved')

100%|██████████| 802383/802383 [00:35<00:00, 22687.54it/s]


Number of non-Canadian entries found: 3382
/home/jovyan/ODBiz/3-Merging/output/nonCanadianEntries.csv saved


In [7]:
pd.unique(non_cad_df.localfile)

array(['ON_Pickering_Business_Directory.csv',
       'BC_Vancouver_Business_Licences.csv',
       'BC_Nanaimo_Business_Licences.csv',
       'ON_York_Region_Business_Directory.csv',
       'ON_Durham_Business_Directory.csv'], dtype=object)

In [8]:
USA_states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'ARIZONA']

foreign_prov_codes = list(set(non_cad_provs) - set(USA_states))
# for i in foreign_prov_codes:
#     print(i)

# Extract a list of indices corresponding to foreign code entries and obtain the subset of the df that contains these entries
foreign_codes_idx = []
for i, row in tqdm(non_cad_df.iterrows(), total = 3382 ):
    if row.province in foreign_prov_codes:
        foreign_codes_idx.append(i)
print(f'\nNumber of foreign entries found: {len(foreign_codes_idx)}')

foreign_codes_df = df.loc[foreign_codes_idx]
pd.options.display.max_columns = None
foreign_codes_df.head()

100%|██████████| 3382/3382 [00:00<00:00, 10863.64it/s]


Number of foreign entries found: 3382


,idx,localfile,business_name,business_sector,business_subsector,business_description,business_id_no,licence_number,licence_type,primary_NAICS,secondary_NAICS,NAICS_descr,alt_econ_act_code,alt_econ_act_descrip,latitude,longitude,full_address,full_address_2,mailing_address,unit,street_no,street_name,street_direction,street_type,city,province,postal_code,country,business_website,email,telephone,telephone_extension,toll_free_telephone,fax,total_no_employees,no_full_time,no_part_time,no_seasonal,date_established,indigenous,status,provider,geo_source,valid_coord,addr_info,has_addr_info
44220,e6e00a859f66014176fe,ON_Pickering_Business_Directory.csv,Scugog Council for the Arts,Council for the arts. Arts resource centre and...,NaN,NaN,99931.0,NaN,NaN,813410.0,NaN,Civic and social organizations,NaN,NaN,44.105207,-78.944945,NaN,NaN,NaN,g--1,1266,Townline,West,Road,Scugog,NaN,L9L1A7,NaN,www.scugogarts.ca,NaN,9059852121.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,Regional Municipality of Durham,Source,True,___g--1_1266_Townline_West_Road_Scugog_L9L1A7,True
44226,72baf502f7054fb7aa98,ON_Pickering_Business_Directory.csv,Forget-Me-Not Alpacas,"Raise Alpacas, Sell Clothing made of Alpaca fu...",NaN,NaN,552799.0,NaN,NaN,112990.0,NaN,All Other Animal Production,NaN,NaN,44.406631,-79.098971,NaN,NaN,NaN,NaN,1595,Brock Concession Rd 3,NaN,NaN,Beaverton,NaN,L0K1A0,NaN,www.forgetmenotalpacas.ca,NaN,2892216102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,Regional Municipality of Durham,Source,True,____1595_Brock Concession Rd 3___Beaverton_L0K1A0,True
44419,54db800e79c02fbbf90e,ON_Pickering_Business_Directory.csv,Elwill Acres,OFA Farm,NaN,NaN,239024.0,NaN,NaN,111999.0,NaN,All other miscellaneous crop farming,NaN,NaN,44.252556,-79.010867,NaN,NaN,NaN,NaN,S13325,Brock Concession Rd 4,NaN,NaN,Sunderland,NaN,L0C1H0,NaN,NaN,NaN,7053572677.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,Regional Municipality of Durham,Source,True,____S13325_Brock Concession Rd 4___Sunderland_...,True
44426,0f602705e8b0c263e154,ON_Pickering_Business_Directory.csv,OFA Farm - B33245,OFA Farm,NaN,NaN,238726.0,NaN,NaN,111999.0,NaN,All other miscellaneous crop farming,NaN,NaN,44.495985,-79.112838,NaN,NaN,NaN,NaN,B33245,Thorah Concession Rd 10,NaN,NaN,Beaverton,NaN,L0K1A0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,Regional Municipality of Durham,Source,True,____B33245_Thorah Concession Rd 10___Beaverton...,True
44440,4237851db447dec2bc0f,ON_Pickering_Business_Directory.csv,Lai & Associates DMC,Digital media production and consultancy.,NaN,NaN,569845.0,NaN,NaN,541890.0,NaN,Other Services Related to Advertising,NaN,NaN,43.838945,-79.083910,NaN,NaN,NaN,NaN,606,The Esplanade,North,NaN,Pickering,NaN,L1V6V2,NaN,www.laiandassociates.ca,NaN,6476463386.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,Regional Municipality of Durham,Source,True,____606_The Esplanade_North__Pickering_L1V6V2,True


In [9]:
# Print the unique countries present
print('--------------------------------------------------------------')
print('Unique Country Entries:')
countries = pd.unique(df['country'])
for i in countries:
        print(i)

--------------------------------------------------------------
Unique Country Entries:
nan
CANADA


In [10]:
sub_df = df[df.province == 'WA']
for i,row in sub_df.iterrows():
    print(row)
    break
# df.loc['idx', '8f6bd3c96413edd33d9d']

In [11]:
# Write the non-cad prov codes to a file
filename = '/home/jovyan/ODBiz/3-Merging/double_check/nonCadProvCodes.txt'
with open(filename, 'w') as f:
    for i in non_cad_provs:
        f.write(f'{i}\n')
print(f'List saved to {filename}')

List saved to /home/jovyan/ODBiz/3-Merging/output/nonCadProvCodes.txt


In [12]:
foreign_prov_codes

['LAKEVIEW ESTATES', '78', nan]

In [13]:
status_lst = pd.unique(df.status)
for i in status_lst:
    print(i)

nan
1.0
OPEN
Issued
Inactive
Cancelled
Pending
Gone Out of Business
Invalid Status Code
APPROVED
Renewal Licensed
Pending Renewal
Licensed
Move in Progress
Renewal Notification Sent
Close in Progress
ISSUED
RENEWAL NOTICE


In [14]:
provinces

array(['BC', 'AB', 'NB', 'ON', nan, 'QC', 'MB', 'SK', 'NS', '78', 'YT',
       'NL', 'LAKEVIEW ESTATES', 'NT', 'NU', 'PE'], dtype=object)